In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datashader
import bokeh
import holoviews
import umap
import umap.plot
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import os
import shutil
from PIL import Image
import copy
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import confusion_matrix as cm
import glob2
from pathlib import Path 
from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d
from scipy.optimize import brentq

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils import tensorboard
import torch.nn.functional as F

from model import StyleEncoder as Encoder 
from data_generator import CatsDataset
from metrics import Accuracy

umap.plot.output_notebook()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Prepare Data

In [ ]:
# Creating Train / Val / Test folders (One time use)

root_dir = 'dataset'
classes_dir = ['black', 'blackwhite', 'gray', 'siberian', 'siamese', 'ginger', 'gingerwhite', 'white', 'other']

val_ratio = 0.2

for cls in classes_dir:
    os.makedirs(root_dir +'/train' + '/' + cls, exist_ok=True)
    os.makedirs(root_dir +'/val' + '/' + cls, exist_ok=True)
    
    # Creating partitions of the data after shuffeling
    src = root_dir + '/' + cls # Folder to copy images from
    
    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames)* (1 - val_ratio))])
    
    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    
    print('Class: ' + cls)
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print()
    
    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'/train/' + cls)
        
        new_path = root_dir +'/train/' + cls + '/' + name.split('/')[-1]
        img = Image.open(new_path)
        img = np.array(img)[:, 256:, :]
        result = Image.fromarray((img).astype(np.uint8))
        result.save(new_path)

    for name in val_FileNames:
        shutil.copy(name, root_dir +'/val/' + cls)
        
        new_path = root_dir +'/val/' + cls + '/' + name.split('/')[-1]
        img = Image.open(new_path)
        img = np.array(img)[:, 256:, :]
        result = Image.fromarray((img).astype(np.uint8))
        result.save(new_path)

### Help Functions

In [ ]:
def display_training_curves(train_epochs, training, val_epochs, validation, title, subplot):
    """
    Plots training and validation curves.
    """
    
    if subplot%10==1:
        plt.subplots(figsize=(5,5), facecolor='#F0F0F0')
        plt.tight_layout()
        
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(train_epochs, training)
    ax.plot(val_epochs, validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'val'])

In [ ]:
def plot_similarity_matrix(matrix, labels_a=None, labels_b=None, ax: plt.Axes=None, title=""):
    """
    Plots similarity matrix.
    """
    
    if ax is None:
        _, ax = plt.subplots()
    fig = plt.gcf()
        
    img = ax.matshow(matrix, extent=(-0.5, matrix.shape[0] - 0.5, 
                                     -0.5, matrix.shape[1] - 0.5))

    ax.xaxis.set_ticks_position("bottom")
    if labels_a is not None:
        ax.set_xticks(range(len(labels_a)))
        ax.set_xticklabels(labels_a, rotation=90)
    if labels_b is not None:
        ax.set_yticks(range(len(labels_b)))
        ax.set_yticklabels(labels_b[::-1])  # Upper origin -> reverse y axis
    ax.set_title(title)

    cax = make_axes_locatable(ax).append_axes("right", size="5%", pad=0.15)
    fig.colorbar(img, cax=cax, ticks=np.linspace(0, 1, 7))
    img.set_clim(0.4, 1)
    img.set_cmap("inferno")
    
    return ax


def plot_cm(targets, predictions, num_classes):
    """
    Plots confusion matrix.
    """
    
    cls_dict = {0: 'black',
                1: 'blackwhite',
                2: 'gray',
                3: 'siberian',
                4: 'siamese',
                5: 'ginger',
                6: 'gingerwhite',
                7: 'white',
                8: 'other',
               }

    confusion_matrix = cm(targets, predictions)
    score = f1_score(targets, predictions, labels=range(num_classes), average='macro')
    precision = precision_score(targets, predictions, labels=range(num_classes), average='macro')
    recall = recall_score(targets, predictions, labels=range(num_classes), average='macro')

    confusion_matrix = np.around(confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis], decimals=2)
    confusion_matrix = pd.DataFrame(confusion_matrix,
                                    index = cls_dict.values(), 
                                    columns = cls_dict.values())


    figure = plt.figure(figsize=(7, 7))
    sns.heatmap(confusion_matrix, annot=True,cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
def adjust_learning_rate(current_iter, optimizer, init_lr, gamma, list_of_iters):
    """
    Learning rate scheduler. Performs exponentially decaying.
    
    :current_iter: int, number of current iteration
    :optimizer: model optimizer
    :init_lr: float, initial learning rate
    :gamma: float, decay rate 
    :list_of_iters: list of ints, iteration numbers for decaying
    
    :return: float, current learning rate
    """
    
    current_lr = 0
    power = 0
    if current_iter < list_of_iters[0]:
        current_lr = init_lr
    elif current_iter > list_of_iters[-1]:
        current_lr = init_lr * (gamma ** len(list_of_iters))
    else:
        list_of_iters.sort(reverse=True)
        nearest_smaller_iter = min(list_of_iters, key=lambda x : x - current_iter > 0 )
        list_of_iters.sort(reverse=False)
        index_of_nearest_smaller_iter = list_of_iters.index(nearest_smaller_iter) 
        power = index_of_nearest_smaller_iter + 1
        current_lr = init_lr * (gamma ** power)
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = current_lr

    return current_lr


def save_checkpoint(model, opt, name):
    """
    Saves current model checkpoint.
        
    :model: model to save
    :opt: optimizer
    :name: str, name of model to save
    """
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": opt.state_dict(),
    }
    torch.save(checkpoint, name)

    
def load_checkpoint(model, checkpoint_dir, optimizer=None):
    """
    Loads model checkpoint.
        
    :model: model to save
    :checkpoints_dir: str, path to the checkpoints
    :optimizer: optimizer
    """
    print(f"Loading checkpoint...")
    checkpoint = torch.load(checkpoint_dir, map_location='cpu')
    model.load_state_dict(checkpoint["state_dict"])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint["optimizer"])
    print('Checkpoint was restored!')

### Model

In [ ]:
s = 5 
m = 0.35
emb_size = 256

model = Encoder(init_channels=32, num_features=emb_size, 
                num_classes=9, dropout=0.4, s=s, m=m).to(device)

model

### Data Generator and Hyperparameters

In [ ]:
initial_learning_rate = 0.001
train_BATCH_SIZE = 64
val_BATCH_SIZE = 1
weight_decay = 1e-4

train_path = glob2.glob('dataset/train/*/*.png')
train_dataset = CatsDataset(train_path, augment=True)
trainloader = DataLoader(train_dataset, batch_size=train_BATCH_SIZE, shuffle=True, drop_last=True)

val_path = glob2.glob('dataset/val/*/*.png')
val_dataset = CatsDataset(val_path, augment=False)
valloader = DataLoader(val_dataset, batch_size=val_BATCH_SIZE, shuffle=False, drop_last=True)

checkpoint_path = Path('models')
model_path = checkpoint_path.joinpath(f'model_s{s}_m{m}_emb{emb_size}.pth')

accuracy = Accuracy()
loss = nn.CrossEntropyLoss()

params = []

for name, values in model.named_parameters():
    if 'bias' not in name and 'norm' not in name:
        params += [{'params': [values], 'lr': initial_learning_rate, 'weight_decay': weight_decay}]
    else:
        params += [{'params': [values], 'lr': initial_learning_rate, 'weight_decay': 0.0}]
        
opt = optim.Adam(params, lr=initial_learning_rate, betas=(0.9, 0.999))

In [ ]:
epochs = 1000
decay_steps=1000
decay_rate=0.9
all_steps = epochs * len(trainloader)

list_of_iters = []
decay_iter = 0

while decay_iter <= all_steps:
    decay_iter = decay_iter + decay_steps
    list_of_iters.append(decay_iter)
    
print(list_of_iters)

## Training

In [ ]:
# train the model

epochs = 1000

train_steps = len(trainloader)
val_steps = len(valloader)

train_epoch_losses=[]
val_epoch_losses=[]

train_epoch_accuracy=[]
val_epoch_accuracy=[]

train_epochs=[]
val_epochs=[]

best_val_acc = -100
best_epoch = 0
last_saved_epoch = 0

for epoch in range(epochs):
    
    train_losses = []
    train_accuracies =[]
    
    for image, label_idx, image_path in trainloader:
    
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)
        
        opt.zero_grad()
        model.train()
        out, out_fcl = model(image, label_idx)
        train_loss = loss(out, label_idx)
        train_loss.backward()
        opt.step()
        
        train_accuracy = accuracy(out, label_idx)
        train_losses.append(train_loss.item())
        train_accuracies.append(train_accuracy)
        
        step = model.get_step()
        last_train_step = copy.deepcopy(step)
        
        current_lr = adjust_learning_rate(step, opt, initial_learning_rate, decay_rate, list_of_iters)
        
    train_epoch_losses.append(np.mean(train_losses))
    train_epoch_accuracy.append(np.mean(train_accuracies))
    train_epochs.append(epoch)
    
    msg = f"| Epoch: {epoch}/{epochs} ({step}/{all_steps}) | Avg Loss: {np.mean(train_losses):#.4} | Avg Accuracy: {np.mean(train_accuracies):#.4} | LR: {current_lr:#.4} | "
    print(msg)
    
    if epoch % 1 ==0:
        val_losses = []
        val_accuracies =[]
        for image, label_idx, image_path in valloader:
            with torch.no_grad():
                
                image = image.to(device, dtype=torch.float)  
                label_idx = label_idx.to(device)
        
                model.eval()
                out, out_fcl = model(image, label_idx)
                val_loss = loss(out, label_idx)
                val_accuracy = accuracy(out, label_idx)
                val_losses.append(val_loss.item())
                val_accuracies.append(val_accuracy)

        msg = f"| Epoch: {epoch}/{epochs} | Eval Avg Loss: {np.mean(val_losses):#.4} | Eval Avg Accuracy: {np.mean(val_accuracies):#.4} | "
        print(msg)

        val_epoch_losses.append(np.mean(val_losses))
        val_epoch_accuracy.append(np.mean(val_accuracies))
        val_epochs.append(epoch)

        model.set_step(last_train_step)           
        
        if val_epoch_accuracy[-1]>=best_val_acc:
            best_val_acc = val_epoch_accuracy[-1]
            best_epoch=epoch
            save_checkpoint(model, opt, model_path)
            
        if epoch - last_saved_epoch > 70:
            break 
            
        if val_epoch_accuracy[-1]>=best_val_acc:
            last_saved_epoch = epoch

In [ ]:
print('Best epoch: ' + str(best_epoch))

### Training Curves

In [ ]:
display_training_curves(train_epochs, train_epoch_losses, val_epochs, val_epoch_losses, 'loss', 211)
display_training_curves(train_epochs, train_epoch_accuracy, val_epochs, val_epoch_accuracy, 'accuracy', 212)

### Validation Accuracy

In [ ]:
# check validation accuracy

model_inference = Encoder(init_channels=32, num_features=emb_size, 
                          num_classes=9, dropout=0.4, s=s, m=m).to(device)

load_checkpoint(model=model_inference, checkpoint_dir=model_path)
accuracy = Accuracy()
loss = nn.CrossEntropyLoss()

val_losses = []
val_accuracies =[]

for image, label_idx, image_path in valloader:
    with torch.no_grad():
                
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)  
        model_inference.eval()
        
        out, out_fcl = model_inference(image, label_idx)
        
        val_loss = loss(out, label_idx)
        val_accuracy = accuracy(out, label_idx)
        val_losses.append(val_loss.item())
        val_accuracies.append(val_accuracy)  
        
print('Validation accuracy: ', np.mean(val_accuracies))

### Tunning

In [ ]:
# tune the model using a low learning rate

initial_learning_rate = 1e-6
train_BATCH_SIZE = 64
val_BATCH_SIZE = 1
weight_decay = 1e-4

epochs = 1000
decay_steps=500
decay_rate=0.9
all_steps = epochs * len(trainloader)

list_of_iters = []
decay_iter = 0

while decay_iter <= all_steps:
    decay_iter = decay_iter + decay_steps
    list_of_iters.append(decay_iter)  
    
print(list_of_iters)

model_tune = Encoder(init_channels=32, num_features=emb_size, num_classes=9, dropout=0.4, s=s, m=m).to(device)
model_tune_path = checkpoint_path.joinpath(f'model_tune_s{s}_m{m}_emb{emb_size}.pth')

params = []

for name, values in model_tune.named_parameters():
    if 'bias' not in name and 'norm' not in name:
        params += [{'params': [values], 'lr': initial_learning_rate, 'weight_decay': weight_decay}]
    else:
        params += [{'params': [values], 'lr': initial_learning_rate, 'weight_decay': 0.0}]

opt = optim.Adam(params, lr=initial_learning_rate, betas=(0.9, 0.999))
load_checkpoint(model=model_tune, checkpoint_dir=model_path, optimizer=opt)
        
accuracy = Accuracy()
loss = nn.CrossEntropyLoss()

In [ ]:
epochs = 1000

train_steps = len(trainloader)
val_steps = len(valloader)

train_epoch_losses=[]
val_epoch_losses=[]

train_epoch_accuracy=[]
val_epoch_accuracy=[]

train_epochs=[]
val_epochs=[]

best_val_acc = -100
best_epoch = 0
last_saved_epoch = 0

model_tune.set_step(0)

for epoch in range(epochs):
    
    train_losses = []
    train_accuracies =[]
    
    for image, label_idx, image_path in trainloader:
    
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)
        
        opt.zero_grad()
        model_tune.train()
        out, out_fcl = model_tune(image, label_idx)
        train_loss = loss(out, label_idx)
        train_loss.backward()
        opt.step()
        
        train_accuracy = accuracy(out, label_idx)
        train_losses.append(train_loss.item())
        train_accuracies.append(train_accuracy)
        
        step = model_tune.get_step()
        last_train_step = copy.deepcopy(step)
        
        # get current learning rate 
        current_lr = adjust_learning_rate(step, opt, initial_learning_rate, decay_rate, list_of_iters)
        
    train_epoch_losses.append(np.mean(train_losses))
    train_epoch_accuracy.append(np.mean(train_accuracies))
    train_epochs.append(epoch)
    
    msg = f"| Epoch: {epoch}/{epochs} ({step}/{all_steps}) | Avg Loss: {np.mean(train_losses):#.4} | Avg Accuracy: {np.mean(train_accuracies):#.4} | LR: {current_lr:#.4} | "
    print(msg)
    
    if epoch % 1 ==0:
        val_losses = []
        val_accuracies =[]
        for image, label_idx, image_path in valloader:
            with torch.no_grad():
                
                image = image.to(device, dtype=torch.float)  
                label_idx = label_idx.to(device)
        
                model_tune.eval()
                out, out_fcl = model_tune(image, label_idx)

                val_loss = loss(out, label_idx)
                val_accuracy = accuracy(out, label_idx)
                val_losses.append(val_loss.item())
                val_accuracies.append(val_accuracy)

        msg = f"| Epoch: {epoch}/{epochs} | Eval Avg Loss: {np.mean(val_losses):#.4} | Eval Avg Accuracy: {np.mean(val_accuracies):#.4} | "
        print(msg)

        val_epoch_losses.append(np.mean(val_losses))
        val_epoch_accuracy.append(np.mean(val_accuracies))
        val_epochs.append(epoch)

        model_tune.set_step(last_train_step)  
            
        if val_epoch_accuracy[-1]>=best_val_acc:
            best_val_acc = val_epoch_accuracy[-1]
            best_epoch=epoch
            save_checkpoint(model_tune, opt, model_tune_path)
            
        if epoch - last_saved_epoch > 70:
            break 
            
        if val_epoch_accuracy[-1]>=best_val_acc:
            last_saved_epoch = epoch

display_training_curves(train_epochs, train_epoch_losses, val_epochs, val_epoch_losses, 'loss', 211)
display_training_curves(train_epochs, train_epoch_accuracy, val_epochs, val_epoch_accuracy, 'accuracy', 212)

### Validation Accuracy

In [ ]:
model_tune_inference = Encoder(init_channels=32, num_features=emb_size, 
                               num_classes=9, dropout=0.4, s=s, m=m).to(device)

load_checkpoint(model=model_tune_inference, checkpoint_dir=model_tune_path)
accuracy = Accuracy()
loss = nn.CrossEntropyLoss()

val_losses = []
val_accuracies =[]

for image, label_idx, image_path in valloader:
    with torch.no_grad():
                
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)  
        model_tune_inference.eval()
        
        out, out_fcl = model_tune_inference(image, label_idx)
        
        val_loss = loss(out, label_idx)
        val_accuracy = accuracy(out, label_idx)
        val_losses.append(val_loss.item())
        val_accuracies.append(val_accuracy)  
        
np.mean(val_accuracies)

### Evaluation

##### U-Map projection for validation data

In [ ]:
vaL_features = []
vaL_labels = []
paths_val = []

for image, label_idx, image_path in valloader:
    with torch.no_grad():    
        
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)
        model_tune_inference.eval()
        
        out, out_fcl = model_tune_inference(image, label_idx)
        out_fcl = out_fcl.cpu()
        out_fcl =  out_fcl / np.linalg.norm(out_fcl, axis=1, keepdims=True)
        
        vaL_features.append(out_fcl[0, :].numpy())
        vaL_labels.append(label_idx.item())
        paths_val.append(image_path)

val_hover_data = pd.DataFrame({'index':paths_val, 'label':vaL_labels})

val_hover_data['item'] = val_hover_data.label.map({0: 'black_val',
                                                   1: 'blackwhite_val',
                                                   2: 'grey_val',
                                                   3: 'siberian_val',
                                                   4: 'siamese_val',
                                                   5: 'ginger_val',
                                                   6: 'gingerwhite_val',
                                                   7: 'white_val',
                                                   8: 'other_val'}
                                                )        
        
vaL_features = np.array(vaL_features)
vaL_labels = np.array(vaL_labels)

In [ ]:
%matplotlib notebook

mapper = umap.UMAP(random_state=42, n_neighbors=10).fit(vaL_features)

p = umap.plot.interactive(mapper, labels=vaL_labels, hover_data=val_hover_data, point_size=3, theme='fire')

umap.plot.show(p)

##### U-Map projection for train data

In [ ]:
train_path = glob2.glob('dataset/train/*/*.png')
train_dataset = CatsDataset(train_path, augment=False)
trainloader = DataLoader(train_dataset, batch_size=1, shuffle=False, drop_last=False)

train_features = []
train_labels = []
paths_train = []

for image, label_idx, image_path in trainloader:
    with torch.no_grad():    
        
        image = image.to(device, dtype=torch.float)  
        label_idx = label_idx.to(device)
        model_tune_inference.eval()
        
        out, out_fcl = model_tune_inference(image, label_idx)
        out_fcl = out_fcl.cpu()
        out_fcl =  out_fcl / np.linalg.norm(out_fcl, axis=1, keepdims=True)
        
        train_features.append(out_fcl[0, :].numpy())
        train_labels.append(label_idx.item())
        paths_train.append(image_path)

train_hover_data = pd.DataFrame({'index':paths_train, 'label':train_labels})
train_hover_data['item'] = train_hover_data.label.map({0: 'black_train',
                                                       1: 'blackwhite_train',
                                                       2: 'grey_train',
                                                       3: 'siberian_train',
                                                       4: 'siamese_train',
                                                       5: 'ginger_train',
                                                       6: 'gingerwhite_train',
                                                       7: 'white_train',
                                                       8: 'other_train'}
                                                    )         
        
train_features = np.array(train_features)
train_labels = np.array(train_labels)

In [ ]:
mapper = umap.UMAP(random_state=42, n_neighbors=10).fit(train_features)

p = umap.plot.interactive(mapper, labels=train_labels, hover_data=train_hover_data, point_size=2, theme='fire')

umap.plot.show(p)

##### U-Map projection for train and validation data

In [ ]:
all_features = np.concatenate((train_features, vaL_features))
all_labels = np.concatenate((train_labels, vaL_labels))
all_hover_data = pd.concat([train_hover_data, val_hover_data], axis=0, ignore_index=True)

mapper = umap.UMAP(random_state=42, n_neighbors=10).fit(all_features)


p = umap.plot.interactive(mapper, labels=all_labels, hover_data=all_hover_data, point_size=2, theme='fire')

umap.plot.show(p)

##### Calculation of the average embeddings 

In [ ]:
black_features = []
black_white_features = []
gray_features = []
siberian_features = []
siamese_features = []
ginger_features =[]
ginger_white_features = []
white_features = []
other_features = []

for i in range(len(train_labels)):
    label = train_labels[i]
    train_feature = train_features[i]
    
    if label == 0:
        black_features.append(train_feature)
        
    elif label == 1:
        black_white_features.append(train_feature)
        
    elif label == 2:
        gray_features.append(train_feature)
        
    elif label == 3:
        siberian_features.append(train_feature)
        
    elif label == 4:
        siamese_features.append(train_feature)
        
    elif label == 5:
        ginger_features.append(train_feature)
        
    elif label == 6:  
        ginger_white_features.append(train_feature)   
        
    elif label == 7:  
        white_features.append(train_feature) 
        
    elif label == 8:  
        other_features.append(train_feature) 

avg_features = [np.mean(black_features, axis = 0)/np.linalg.norm(np.mean(black_features, axis = 0), 2), 
                np.mean(black_white_features, axis = 0)/np.linalg.norm(np.mean(black_white_features, axis = 0), 2), 
                np.mean(gray_features, axis = 0)/np.linalg.norm(np.mean(gray_features, axis = 0), 2), 
                np.mean(siberian_features, axis = 0)/np.linalg.norm(np.mean(siberian_features, axis = 0), 2), 
                np.mean(siamese_features, axis = 0)/np.linalg.norm(np.mean(siamese_features, axis = 0), 2), 
                np.mean(ginger_features, axis = 0)/np.linalg.norm(np.mean(ginger_features, axis = 0), 2), 
                np.mean(ginger_white_features, axis = 0)/np.linalg.norm(np.mean(ginger_white_features, axis = 0), 2),
                np.mean(white_features, axis = 0)/np.linalg.norm(np.mean(white_features, axis = 0), 2),
                np.mean(other_features, axis = 0)/np.linalg.norm(np.mean(other_features, axis = 0), 2)]  

##### Evaluation using cosine similarity

In [ ]:
y_train_pred =[]

for i in range(len(train_labels)):
    y_train_pred.append(np.argmax((np.inner(train_features[i], avg_features))))


accuracy = accuracy_score(train_labels, y_train_pred)
print("Train accuracy score: " + str(accuracy))

In [ ]:
y_val_pred =[]

for i in range(len(vaL_labels)):
    y_val_pred.append(np.argmax((np.inner(vaL_features[i], avg_features))))

accuracy = accuracy_score(vaL_labels, y_val_pred)
print("Validation accuracy score: " + str(accuracy))

##### Similarity matrix

In [ ]:
plot_cm(vaL_labels, y_val_pred, 9)

##### Estimate Equal Error Rate

In [ ]:
predictions = []
labels = []

num_enrollment_classes = 9
enrollment_mean_embeddings = avg_features
trial_test_embeddings = vaL_features
trial_test_targets = vaL_labels

for i in range(num_enrollment_classes):
    for j in range(len(trial_test_embeddings)):
        
        e1 = enrollment_mean_embeddings[i]
        e2 = trial_test_embeddings[j]
        similarity = np.inner(e1, e2)  
        predictions.append(similarity)
        
        if i == trial_test_targets[j]:
            labels.append(1)
        else:
            labels.append(0)
            
fpr, tpr, thresholds = roc_curve(labels, predictions, pos_label=1)
f = interp1d(fpr, tpr, kind = 'nearest')

x_new = np.arange(0, 1, 0.001)
y_new = f(x_new)

eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr, kind = 'nearest')(x), 0., 1., xtol=1e-12, maxiter=1000)

print('EER, [%]: ' + str(eer*100))

plt.figure(figsize=(7,7))
plt.plot(fpr, tpr, '.', color='k', markersize=2)
plt.plot(x_new, y_new,  '-', color='C0', linewidth=1)
plt.plot([1, eer], [0, 1-eer], '-', color='r')
plt.ylabel('True positive rate', size=14)
plt.xlabel('False positive rate', size=14);
plt.show()